In [ ]:
library(dplyr)

In [ ]:
df <- read.csv("BuySell.csv", header = TRUE, sep = ",")
nrow(df)
head(df,3)

In [ ]:
# Convert Excel dates when in the 1900 date format

df$date <- as.Date(df$date, origin = "1899-12-30")
#df$date_s <- as.Date(df$date_s, origin = "1899-12-30")
head(df, 3)

In [ ]:
# First allocate, then play with cost basis methods

adj_tally_q <- function(tally_q, proceeds) {
    # cat("tally_q in the remote fct", tally_q, "\n")
    tally_q <- tally_q + proceeds
    cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
    return(tally_q)        
    }

## <font color = yellow> Keeper! These values agree with .csv/spreadsheet
    
##### Getting the spreadsheet right w Buy/Sells remains the workout

In [ ]:
# Function that reads BuySell.csv and will perform actions which agree with the values of (tally_q_q) on Sell dates

make_buysell_csv <- function() {    

    # remote fct to test passing value from fct to remote to this remote
     adj_basis <- function(tally_q, proceeds) {
        print(paste0("In adj_basis fct. tally_q is: ", tally_q, " Sell is: ", proceeds))
        n <- 2
        tally_q <<- tally_q + n
        return(tally_q)
        }   

    # remote fct to adjust the running stock tally_q (combine sum of Buys with the next Sell), return adjusted tally_q
    adj_tally_q <- function(tally_q, proceeds) {
        cat("tally_q in the remote fct", tally_q, "proceeds is: ", proceeds, "\n")
        tally_q <- tally_q + proceeds
        cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
        # Fct call
        # adjusted <- adj_basis(tally_q, proceeds)
        # print(paste0("adjusted from remote fct is: ", adjusted))
        return(tally_q)        
        }       

    # Main body     

    # Create counter vars
    ct <- 0
    tally_q <<- 0
    row_ct <<- 0
    proceeds <- 0
    
    cat("The df has the following nrows: ", nrow(df), "\n \n")

    # loop through the df
    for(i in 1:nrow(df))  { 

         ct <- ct + 1
        # Sum the buy values from each row
        if(df[i, 'quan'] > 0 ) {
            row_ct <- row_ct + 1
           # tally_q the buys from each row 
            tally_q <- tally_q + df[i, 'quan']
            cat("BUY: Row no: ", row_ct, "Cost is: ", df[i, "basis"], ", tally_q is:", tally_q, "\n")
            
            # cat("BUY ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q, "\n")
            }

            #  When a Sell occurs, Call remote fct to adjust the tally_q == tally_q_q
            if(df[i, 'quan'] < 0 )  {
                proceeds <- df[i, 'quan']
                row_ct <- row_ct + 1
               cat("SELL ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q, " Sell Proceeds: ", df[i, 'basis'], "\n" )
               tally_q <- adj_tally_q(tally_q, proceeds) 
                }
       }  # end For-loop 

        cat("\n")
        cat("The FINAL tally_q is :", tally_q, "\n")
        ct
} # end fct

#  These value will agree with the spreadsheet: BuySell.csv / .xlsx

In [ ]:
make_buysell_csv()

In [ ]:
# stopper

In [ ]:
    # remote fct to adjust the running stock tally_q (combine sum of Buys with the next Sell), return adjusted tally_q
    adj_tally_q <- function(tally_q, proceeds) {
        cat("tally_q in the remote fct", tally_q, "The proceeds is: ", proceeds, "\n")
        tally_q <- tally_q + proceeds
        cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
       
        return(tally_q)        
        }       


In [ ]:
# Identify the rows in the df where the proceedss occur

# Create counter vars
ct <- 0
tally_q <- 0
row_ct <<- 1
cat("The df has the following nrows: ", nrow(df), "\n \n")

# loop through the df
for(i in 1:nrow(df))  { 
    
     ct <- ct + 1
    # Sum the buy values from each row
    if(df[i, 'quan'] > 0 ) {
        
        # tally_q the buys from each row 
         tally_q <- tally_q + df[i, 'quan']
        cat("BUY. ", df[i,'quan'], " row number is: ", row_ct, " tally_q is: ", tally_q, "\n")
       
        row_ct <- row_ct + 1
        #print(paste0("BUY ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q))       
        }
    
        
        #  When a Sell occurs, Call remote fct to adjust the tally_q == tally_q_q
        if(df[i, 'quan'] < 0 )  {
            proceeds <- df[i, 'quan']
            row_ct <- row_ct + 1
           cat("SELL ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q, " Sell Proceeds: ", df[i, 'basis'], "\n" )
           tally_q <- adj_tally_q(tally_q, proceeds) 
            }
   }
 
cat("\n")
cat("The FINAL tally_q is :", tally_q, "\n")  # This value s/b 0 
ct

#  These value agree with the spreadsheet (tally_q_q on SELL line) See: BuySell.csv / .xlsx
#  These value agree with the spreadsheet: BuySell.csv / .xlsx

## <font color = yellow> Convert the entire thing into a fct

In [ ]:
 # adj_basis <- function(tally_q, proceeds) {
 #        print(paste0("In adj_basis fct. tally_q is: ", tally_q, " Sell is: ", proceeds))
 #        n <- 2
 #        tally_q <<- tally_q + n
 #        return(tally_q)
 #        }

In [ ]:
# Function that reads BuySell.csv and will perform actions which agree with the values of (tally_q_q) on Sell dates

make_buysell_csv <- function() {    

    # remote fct to test passing value from fct to remote to this remote
     adj_basis <- function(tally_q, proceeds) {
        print(paste0("In adj_basis fct. tally_q is: ", tally_q, " Sell is: ", proceeds))
        n <- 2
        tally_q <<- tally_q + n
        return(tally_q)
        }   

    # remote fct to adjust the running stock tally_q (combine sum of Buys with the next Sell), return adjusted tally_q
    adj_tally_q <- function(tally_q, proceeds) {
        # cat("tally_q in the remote fct", tally_q, "\n")
        tally_q <- tally_q + proceeds
        # cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
        # Fct call
        # adjusted <- adj_basis(tally_q, proceeds)
        # print(paste0("adjusted from remote fct is: ", adjusted))
        return(tally_q)        
        }       

    # Main body     

    # Create counter vars
    ct <- 0
    tally_q <<- 0
    tally_sales <<- 0
    row_ct <<- 0
    cat("The df has the following nrows: ", nrow(df), "\n \n")

    # loop through the df
    for(i in 1:nrow(df))  { 

         ct <- ct + 1
        # Sum the buy values from each row
        if(df[i, 'quan'] > 0 ) {
            row_ct <- row_ct + 1
           # tally_q the buys from each row 
            tally_q <- tally_q + df[i, 'quan']
            #cat("BUY: Row no: ", row_ct, " Cost ", df[i, "basis"], " tally_q is: ", tally_q, "\n")
            
            # print(paste0("BUY ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q))       
            }

            #  When a Sell occurs, Call remote fct to adjust the tally_q == tally_q_q
            if(df[i, 'quan'] < 0 )  {
                proceeds <- df[i, 'quan']
                row_ct <- row_ct + 1
                tally_sales <<- df[i, 'basis'] + tally_sales
                #cat("tally_sales is: ", tally_sales, "\n")
                cat("SELL ", df[i,'quan'], " Row number is: ", row_ct, " Sale quan is :", proceeds, " Sell Proceeds: ", df[i, 'basis'], "\n" )
                tally_q <- adj_tally_q(tally_q, proceeds) 
                }       
        
       }  # end For-loop 

        cat("\n")
        cat("The FINAL tally_q is :", tally_q, "\n")
        cat("The FINAL tally_sales is :", tally_sales, "\n")
        ct
} # end fct

#  These value will agree with the spreadsheet: BuySell.csv / .xlsx

In [ ]:
#  Call the fct
make_buysell_csv()
#  These value will agree with the spreadsheet: BuySell.csv / .xlsx

In [ ]:
# stopper

## <font color = lime> New Addition: Add new tally_c var to track cost basis

In [ ]:
# remote fct to adjust the running Buy tally_q with the next Sell quan
#  & return adjusted tally_q
    adj_tally_q <- function(tally_q, proceeds) {
        tally_q <- tally_q + proceeds
        cat("ADJUSTED tally_q - reduced by proceeds in remote fct", tally_q, "\n")
        return(tally_q)        
        }       

In [ ]:
# Identify the rows in the df where the proceedss occur

# Create counter vars
ct <- 0
tally_q <- 0
row_ct <<- 1
accum_buy_basis <<- 0
accum_sell_basis <<- 0


cat("The df has the following nrows: ", nrow(df), "\n \n")

# loop through the df
for(i in 1:nrow(df))  { 
    
    ct <- ct + 1
    # Loop if Buy value larger than zero
    if(df[i, 'quan'] > 0 ) {        
       # tally the Buy from each row 
       tally_q <- tally_q + df[i, 'quan']
       cat("BUY. ", df[i,'quan'], " row number is: ", row_ct, " tally_q is: ", tally_q, "\n") 
        accum_buy_basis <- accum_buy_basis + df[i, 'basis']
       row_ct <- row_ct + 1
       }
              
    #  If row is a Sell event, execute remote fct and reduce tally_q by Sell quan
    if(df[i, 'quan'] < 0 )  {
       proceeds <- df[i, 'quan']
       row_ct <- row_ct + 1
       cat("SELL ", df[i,'quan'], " Row number is: ", row_ct, " The tally_q is: ", tally_q, " Sell Proceeds: ", df[i, 'basis'], "\n" )
        accum_sell_basis <- accum_sell_basis + df[i, 'basis']
       # fct call
       tally_q <- adj_tally_q(tally_q, proceeds) 
       }    
   }
 
cat("\n")
cat("The FINAL tally_q is :", tally_q, "\n")
ct

accum_buy_basis
accum_sell_basis
unadjusted_pl <- c(accum_buy_basis - abs(accum_sell_basis))
unadjusted_pl


#  These value agree with the spreadsheet (tally_q_q on SELL line) See: BuySell.csv / .xlsx

In [ ]:
df

In [ ]:
sum(df$basis)